In [1]:
import re
import pandas as pd
import numpy as np
import ast
import operator



def union_num_index_syn(dic_syn):
    _dic_syn_=dic_syn.copy()
    for row,dic in enumerate(_dic_syn_):
        synset=set()
        for syn in dic.values():
            synset=synset.union(set(syn))
        _dic_syn_[row]=synset
    return(_dic_syn_)    


def union_num_index_syn_rule_out(dic_syn,list_index_rule_out):
    _dic_syn_=dic_syn.copy()
    for row,dic in enumerate(_dic_syn_):
        synset=set()
        for idx,syn in dic.items():
            if idx in list_index_rule_out[row]:
                synset=synset.union(set(syn))
        _dic_syn_[row]=synset
    return(_dic_syn_) 

    
def union_syns(dataset,features,union_feature):
    _dataset_=dataset.copy()
    _dataset_[union_feature]=""
    for row in range(len(_dataset_[union_feature])):
        synset=set()
        for feature in features:
            synset=synset.union(set(_dataset_[feature][row]))
        _dataset_[union_feature][row]=synset
    return(_dataset_[union_feature])


def difference_cols(col_1,col_2):
    col_1_=col_1.copy()
    for row in range(0,len(col_1)):
        set_1=col_1[row]
        set_2=col_2[row]
        col_1_[row]=set_1.difference(set_2)
    return(col_1_)

def synonym_voter(dataset,features,union_and_removed_feature):
    _dataset_=dataset.copy()
    _dataset_[union_and_removed_feature]=_dataset_[union_and_removed_feature].apply(lambda s: list(s))
    _dataset_["temp"]=""
    for row in range(0,len(_dataset_[union_and_removed_feature])):
        ranked_dic={}
        for value in (_dataset_[union_and_removed_feature][row]):
            for feature in features:
                if value in _dataset_[feature][row]:
                    if value in ranked_dic:
                        ranked_dic[value]+=1
                    else:
                        ranked_dic[value]=1
        ranked_dic = sorted(ranked_dic.items(), key=operator.itemgetter(1),reverse=True)
        _dataset_["temp"][row]=ranked_dic
    return(_dataset_["temp"])
        

In [3]:
words_table=pd.read_excel(
    "/Users/mehdi.mirzapour/Dropbox/PractiKPharma/PractiKPharma_Code/excels/2_Phase3_Word_Sense_Extraction.xlsx")

columns=['JDM_Sense',
         'JDM',
         'WordNet',
         'Cnrtl',
         'Synonymo',
         'Cisco',
         'DicSyn_Sense']


words_table=words_table.fillna("[]")
words_table["DicSyn_Sense"]=words_table["DicSyn_Sense"].apply(lambda s: re.sub("\[\]", '{}', s))
words_table["JDM_Sense"]=words_table["JDM_Sense"].apply(lambda s: re.sub("\[\]", '{}', s))

for col in columns:
    words_table[col]=words_table[col].apply(lambda s: re.sub(",[\s]*nan", ' ', s))
    words_table[col]=words_table[col].apply(lambda s: re.sub("[^,\s]*nan[\s]+,", '[', s))
    words_table[col]=words_table[col].apply(lambda s: ast.literal_eval(s))

words_table["DicSyn_Sense_"]=union_num_index_syn(words_table["DicSyn_Sense"])  
words_table["JDM_Sense_"]=union_num_index_syn(words_table["JDM_Sense"])  

words_table["Union_Sense"]=union_syns(words_table,
                                      ['JDM_Sense_','JDM','WordNet','Cnrtl','Synonymo','Cisco','DicSyn_Sense_']
                                      ,"Union_Sense")

words_table["DicSyn_Sense_Filter"]="[]"
words_table["DicSyn_Sense_Filter"]=words_table["DicSyn_Sense_Filter"].apply(lambda s: ast.literal_eval(s))

words_table["DicSyn_Sense_to_be_del"]=union_num_index_syn_rule_out(words_table["DicSyn_Sense"],words_table["DicSyn_Sense_Filter"])  

words_table["JDM_Sense_"]=words_table["Union_Sense"].apply(lambda s: s.difference())
words_table["Final_List"]=difference_cols(words_table["Union_Sense"],words_table["DicSyn_Sense_to_be_del"])

words_table["Synonyms_Voting"]=synonym_voter(words_table,
              ['JDM_Sense_','JDM','WordNet','Cnrtl','Synonymo','Cisco','DicSyn_Sense_'],
              "Final_List")

words_table_to_write=words_table[['Word','Word_Corrected','JDM_Sense','JDM','WordNet','Cnrtl','Synonymo','Cisco','DicSyn_Sense','Synonyms_Voting']]


words_table_to_write["Synonyms_Voting_Top5"]=""
words_table_to_write["Synonyms_Voting_Top10"]=""
words_table_to_write["Synonyms_Voting_Top15"]=""

for row in range(len(words_table_to_write["Synonyms_Voting"])):
    words_table_to_write["Synonyms_Voting_Top5"][row]=[x for x,y in words_table_to_write["Synonyms_Voting"][row][0:5]]
    words_table_to_write["Synonyms_Voting_Top10"][row]=[x for x,y in words_table_to_write["Synonyms_Voting"][row][0:10]]
    words_table_to_write["Synonyms_Voting_Top15"][row]=[x for x,y in words_table_to_write["Synonyms_Voting"][row][0:15]]



words_table_to_write.to_excel(
    "/Users/mehdi.mirzapour/Dropbox/PractiKPharma/PractiKPharma_Code/excels/3_Phase3_Word_Sense_Extraction.xlsx")

words_table_to_write  

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is tryin

,Word,Word_Corrected,JDM_Sense,JDM,WordNet,Cnrtl,Synonymo,Cisco,DicSyn_Sense,Synonyms_Voting,Synonyms_Voting_Top5,Synonyms_Voting_Top10,Synonyms_Voting_Top15
0,suffisant,suffisant,"{0: ['assez', 'acceptable', 'satisfaisant', 'r...","[important, vaniteux, correct, crâneur, raison...","[adéquat, suffisant, bégueule, collet monté, p...","[pourvoir, satisfaire, apaiser, baster, conten...","[acceptable, affecté, approuvable, arrogant, a...","[vaniteux, fat, orgueilleux, prétentieux, prés...","{0: ['arrogant', 'insolent', 'orgueilleux', 'p...","[(orgueilleux, 5), (arrogant, 5), (prétentieux...","[orgueilleux, arrogant, prétentieux, passable,...","[orgueilleux, arrogant, prétentieux, passable,...","[orgueilleux, arrogant, prétentieux, passable,..."
1,preuve,preuve,{},"[certificat, signe, raisonnement, document, il...","[corroboration, preuve, validation, salon, exe...","[témoignage, justification, marque, signe, ind...","[échantillon, affirmation, épreuve judiciaire,...","[témoignage, justification, indice, démonstrat...","{0: ['fait', 'charge', 'document', 'justificat...","[(témoignage, 7), (indice, 7), (confirmation, ...","[témoignage, indice, confirmation, conviction,...","[témoignage, indice, confirmation, conviction,...","[témoignage, indice, confirmation, conviction,..."
2,nié,nié,{},"[désavoué, rejeté, dénié, renié, démenti, disc...",[],"[contester, contredire, désavouer, refuser, ré...","[confirmé, contesté, contester, contredire, co...",[],{},"[(désavoué, 3), (dénié, 3), (réfuter, 3), (con...","[désavoué, dénié, réfuter, contredit, renier]","[désavoué, dénié, réfuter, contredit, renier, ...","[désavoué, dénié, réfuter, contredit, renier, ..."
3,cause,cause,{},"[origine, agent, occasion, facteur, raison, ar...","[cause, chose, objet, raison, campagne, effort...","[motif, principe, origine, raison, germe, fond...","[action, affaire, agent, élément, artisan, éti...","[motif, germe, principe, fondement, origine, p...","{0: ['affaire', 'dossier', 'procès', 'sujet', ...","[(matière, 7), (motif, 7), (raison, 7), (ferme...","[matière, motif, raison, ferment, procès]","[matière, motif, raison, ferment, procès, suje...","[matière, motif, raison, ferment, procès, suje..."
4,source,source,"{0: ['hydrologie', 'fontaine']}","[base, agent, provenance, puits, foyer, fontai...","[source, graine, semence, puits, caractère, fa...","[origine, principe, racine, fondement, germe, ...","[agent, amont, étymologie, base, canal, centre...","[origine, racine, principe, germe, ferment, pr...","{0: ['commencement', 'début', 'origine', 'prin...","[(origine, 7), (puits, 7), (racine, 6), (prove...","[origine, puits, racine, provenance, germe]","[origine, puits, racine, provenance, germe, pr...","[origine, puits, racine, provenance, germe, pr..."
5,problèmes,problème,"{0: ['difficulté', 'os', 'difficulté']}","[souci, complication, colle, énigme, ennui, qu...","[difficulté, effort, peine, problème, délivran...","[difficulté, ennui, question, affaire, histoir...","[affaire, énigme, colle, complication, danger,...","[question, couac, énigme, colle, difficulté, t...","{0: ['fait', 'difficulté', 'ennui', 'question'...","[(question, 7), (difficulté, 7), (affaire, 6),...","[question, difficulté, affaire, ennui, sujet]","[question, difficulté, affaire, ennui, sujet, ...","[question, difficulté, affaire, ennui, sujet, ..."
6,raison,raison,"{0: ['entendement', 'entendement'], 1: ['motif...","[cause, jugeote, logique, intellect, considéra...","[cause, raison, cas, complet, ensemble, poursu...","[esprit, sens, intelligence, sens, entendement...","[allégation, équité, argument, argumentation, ...","[entendement, motif, esprit, intelligence, jug...","{0: ['allégation', 'justification', 'excuse', ...","[(motif, 7), (cause, 7), (modération, 6), (int...","[motif, cause, modération, intelligence, enten...","[motif, cause, modération, intelligence, enten...","[motif, cause, modération, intelligence, enten..."
7,nie,nie,{},[],[],"[contester, contredire, désavouer, refuser, ré.

In [3]:
words_table["JDM_Sense_Filters"]=[0]for i in ['JDM_Sense_','JDM','French_WordNet','Cnrtl','Synonymo','Cisco','DicSyn_Sense_']:
    print([x for x in words_table["JDM"][0] if x not in words_table["Union_Sense"][0]]==[])

True
True
True
True
True
True
True
